In [6]:
# Import packages

import warnings
warnings.filterwarnings("ignore")

from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
import numpy as np

In [7]:
# Read in train data

f = open("train.txt")
line = f.readline()
line = f.readline()
X_train = []
Y_train = []
while line:
    eles = line.split()
    X_train.append([float(eles[1]), float(eles[2]), float(eles[3]), float(eles[4])])
    Y_train.append(int(eles[0]))
    line = f.readline()
f.close()     
X_train = np.array(X_train)
Y_train = np.array(Y_train)
print(X_train.shape)
print(Y_train.shape)

# Read in test data

f = open("test.txt")
line = f.readline()
line = f.readline()
X_test = []
Y_test = []
while line:
    eles = line.split()
    X_test.append([float(eles[1]), float(eles[2]), float(eles[3]), float(eles[4])])
    Y_test.append(int(eles[0]))
    line = f.readline()
X_test = np.array(X_test)
Y_test = np.array(Y_test)
f.close() 
print(X_test.shape)
print(Y_test.shape)


(120, 4)
(120,)
(30, 4)
(30,)


In [29]:
# Helper Functions

zero_class = np.where(Y_train==0)
one_class = np.where(Y_train==1)
two_class = np.where(Y_train==2)

Y_0 = Y_train.copy()
Y_0[zero_class] = -1    # type: ignore
Y_0[one_class] = 1      # type: ignore
Y_0[two_class] = 1      # type: ignore

Y_1 = Y_train.copy()
Y_1[zero_class] = 1     # type: ignore
Y_1[one_class] = -1     # type: ignore
Y_1[two_class] = 1      # type: ignore

Y_2 = Y_train.copy()
Y_2[zero_class] = 1     # type: ignore
Y_2[one_class] = 1      # type: ignore
Y_2[two_class] = -1     # type: ignore

def array2str(array):
    s = ""
    # print(array)
    for ele in array:
        s = s + str(ele) + ", "
    # print(s)
    return s[:-2]

def oneVsRest(kernel, C, degree = 3, gamma = 1):
    clf0 = SVC(kernel=kernel, C=C, random_state=0, probability=True, degree=degree, gamma=gamma)
    clf0.fit(X_train, Y_0)

    clf1 = SVC(kernel=kernel, C=C, random_state=0, probability=True, degree=degree, gamma=gamma)
    clf1.fit(X_train, Y_1)

    clf2 = SVC(kernel=kernel, C=C, random_state=0, probability=True, degree=degree, gamma=gamma)
    clf2.fit(X_train, Y_2)

    return clf0, clf1, clf2

def errorCalculation(clf0, clf1, clf2):
    N = X_train.shape[0]
    error = 0
    p0 = clf0.predict_proba(X_train)
    p1 = clf1.predict_proba(X_train)
    p2 = clf2.predict_proba(X_train)
    for i in range(N):
        if p0[i][0] > p1[i][0] and p0[i][0] > p2[i][0]:
            if Y_train[i] != 0:
                error += 1
                print("Predict: 0, True:", Y_train[i])
        elif p1[i][0] > p0[i][0] and p1[i][0] > p2[i][0]:
            if Y_train[i] != 1:
                error += 1
                print("Predict: 1, True:", Y_train[i])
        elif p2[i][0] > p0[i][0] and p2[i][0] > p1[i][0]:
            if Y_train[i] != 2:
                error += 1
                print("Predict: 2, True:", Y_train[i])
    train_error = error / N

    # Test Error Calculation
    N = X_test.shape[0]
    error = 0
    p0 = clf0.predict_proba(X_test)
    p1 = clf1.predict_proba(X_test)
    p2 = clf2.predict_proba(X_test)
    for i in range(N):
        if p0[i][0] > p1[i][0] and p0[i][0] > p2[i][0]:
            if Y_test[i] != 0:
                error += 1
                print("Predict: 0, True:", Y_test[i])
        elif p1[i][0] > p0[i][0] and p1[i][0] > p2[i][0]:
            if Y_test[i] != 1:
                error += 1
                print("Predict: 1, True:", Y_test[i])
        elif p2[i][0] > p0[i][0] and p2[i][0] > p1[i][0]:
            if Y_test[i] != 2:
                error += 1
                print("Predict: 2, True:", Y_test[i])
    test_error = error / N

    return train_error, test_error

def slackCalculation(clf, X, Y):
    N = X.shape[0]
    pred = clf.predict(X)
    slack = []
    for i in range(N):
        d = 0
        if Y[i] == -1:
            d = (1 + (np.dot(clf.coef_, X[i]) + clf.intercept_)) / np.linalg.norm(clf.coef_)
            # print("Pred: ", pred[i], ", True: ", Y[i], ", slack: ", d)
            
        elif Y[i] == 1:
            d = (1 - (np.dot(clf.coef_, X[i]) + clf.intercept_)) / np.linalg.norm(clf.coef_)
            # print("Pred: ", pred[i], ", True: ", Y[i], ", slack: ", d)
        d = max(0, d[0])
        slack.append(d) 
    return slack



In [9]:
# Problem 1: standard SVM model

# Adjustable Parameters
C = 1e5
kernel = "linear"
        
# Standard SVM Model Construction
clf0, clf1, clf2 = oneVsRest(kernel, C)

# Error Calculation
train_error, test_error = errorCalculation(clf0, clf1, clf2)

# Output
f = open("SVM_linear.txt", "w")
f.write(str(train_error) + "\n")            # training_error
f.write(str(test_error) + "\n")             # testing_error
f.write(array2str(clf0.coef_[0]) + "\n")    # w_of_setosa
f.write(str(clf0.intercept_[0]) + "\n")     # b_of_setosa
f.write(array2str(clf0.support_) + "\n")    # support_vector_indice_of_setosa
f.write(array2str(clf1.coef_[0]) + "\n")    # w_of_versicolor
f.write(str(clf1.intercept_[0]) + "\n")     # b_of_versicolor
f.write(array2str(clf1.support_) + "\n")    # support_vector_indice_of_versicolor
f.write(array2str(clf2.coef_[0]) + "\n")    # w_of_virginica
f.write(str(clf2.intercept_[0]) + "\n")     # b_of_virginica
f.write(array2str(clf2.support_) + "\n")    # support_vector_indice_of_virginica

f.close()



Predict: 2, True: 1
Predict: 2, True: 1
Predict: 2, True: 1
Predict: 2, True: 1
Predict: 2, True: 1
Predict: 2, True: 1
Predict: 2, True: 1
Predict: 2, True: 1
Predict: 2, True: 1
Predict: 2, True: 1
Predict: 1, True: 2
Predict: 1, True: 2
Predict: 1, True: 2
Predict: 2, True: 1
Predict: 2, True: 1
Predict: 2, True: 1
Predict: 2, True: 1


In [31]:
# Problem 2: SVM model with slack variables

# Adjustable Parameters
kernel = "linear"

# 10 Experiments
f = open("SVM_slack.txt", "w")
for i in range(1, 11):
    C = i * 0.1
        
    # SVM Model Construction
    clf0, clf1, clf2 = oneVsRest(kernel, C)

    # Error Calculation
    train_error, test_error = errorCalculation(clf0, clf1, clf2)
    
    # Slack Calculation
    s0 = slackCalculation(clf0, X_train, Y_0)
    s1 = slackCalculation(clf1, X_train, Y_1)
    s2 = slackCalculation(clf2, X_train, Y_2)
    
    # Output
    f.write("C = " + str(C) + "\n")
    f.write(str(train_error) + "\n")            # training_error
    f.write(str(test_error) + "\n")             # testing_error
    f.write(array2str(clf0.coef_[0]) + "\n")    # w_of_setosa
    f.write(str(clf0.intercept_[0]) + "\n")     # b_of_setosa
    f.write(array2str(clf0.support_) + "\n")    # support_vector_indice_of_setosa
    f.write(array2str(s0) + "\n")               # slack_variable_of_setosa
    f.write(array2str(clf1.coef_[0]) + "\n")    # w_of_versicolor
    f.write(str(clf1.intercept_[0]) + "\n")     # b_of_versicolor
    f.write(array2str(clf1.support_) + "\n")    # support_vector_indice_of_versicolor
    f.write(array2str(s1) + "\n")               # slack_variable_of_versicolor
    f.write(array2str(clf2.coef_[0]) + "\n")    # w_of_virginica
    f.write(str(clf2.intercept_[0]) + "\n")     # b_of_virginica
    f.write(array2str(clf2.support_) + "\n")    # support_vector_indice_of_virginica
    f.write(array2str(s2) + "\n\n")             # slack_variable_of_virginica
f.close()

Predict: 2, True: 1
Predict: 2, True: 1
Predict: 2, True: 1
Predict: 2, True: 1
Predict: 1, True: 2
Predict: 2, True: 1
Predict: 2, True: 1
Predict: 2, True: 1
Predict: 1, True: 2
Predict: 2, True: 1
Predict: 2, True: 1
Predict: 2, True: 1
Predict: 1, True: 2
Predict: 2, True: 1
Predict: 2, True: 1
Predict: 1, True: 2
Predict: 2, True: 1
Predict: 2, True: 1
Predict: 2, True: 1
Predict: 1, True: 2
Predict: 2, True: 1
Predict: 2, True: 1
Predict: 1, True: 2
Predict: 2, True: 1
Predict: 2, True: 1
Predict: 2, True: 1
Predict: 1, True: 2
Predict: 2, True: 1
Predict: 2, True: 1
Predict: 1, True: 2
Predict: 2, True: 1
Predict: 2, True: 1
Predict: 2, True: 1
Predict: 1, True: 2
Predict: 1, True: 2
Predict: 2, True: 1
Predict: 2, True: 1
Predict: 1, True: 2
Predict: 2, True: 1
Predict: 2, True: 1
Predict: 2, True: 1
Predict: 2, True: 1
Predict: 1, True: 2
Predict: 1, True: 2
Predict: 2, True: 1
Predict: 1, True: 2
Predict: 2, True: 1
Predict: 2, True: 1
Predict: 2, True: 1
Predict: 2, True: 1


In [11]:
# Problem 3: SVM model with kernal functions and slack variables

def modelAnalysis(clf0, clf1, clf2, filename):
    # Error Calculation
    train_error, test_error = errorCalculation(clf0, clf1, clf2)

    # Output
    f = open(filename, "w")
    f.write(str(train_error) + "\n")            # training_error
    f.write(str(test_error) + "\n")             # testing_error
    f.write(str(clf0.intercept_[0]) + "\n")     # b_of_setosa
    f.write(array2str(clf0.support_) + "\n")    # support_vector_indice_of_setosa
    f.write(str(clf1.intercept_[0]) + "\n")     # b_of_versicolor
    f.write(array2str(clf1.support_) + "\n")    # support_vector_indice_of_versicolor
    f.write(str(clf2.intercept_[0]) + "\n")     # b_of_virginica
    f.write(array2str(clf2.support_) + "\n")    # support_vector_indice_of_virginica
    f.close()

# Adjustable Parameters
C = 1

# 2nd-order polynomial kernel
clf0, clf1, clf2 = oneVsRest(kernel="poly", C=C, degree=2)
modelAnalysis(clf0, clf1, clf2, "SVM_poly2.txt")

# 3nd-order polynomial kernel
clf0, clf1, clf2 = oneVsRest(kernel="poly", C=C)
modelAnalysis(clf0, clf1, clf2, "SVM_poly3.txt")
    
# Radial Basis Function kernel
clf0, clf1, clf2 = oneVsRest(kernel="rbf", C=C)
modelAnalysis(clf0, clf1, clf2, "SVM_rbf.txt")

# Sigmoidal kernel
clf0, clf1, clf2 = oneVsRest(kernel="sigmoid", C=C)
modelAnalysis(clf0, clf1, clf2, "SVM_sigmoid.txt")


Predict: 2, True: 1
Predict: 2, True: 1
Predict: 2, True: 1
Predict: 1, True: 2
Predict: 2, True: 1
Predict: 2, True: 1
Predict: 2, True: 1
Predict: 2, True: 1
Predict: 1, True: 2
